In [1]:
import sys
import tensorflow as tf
sys.path.append("/visuworks/Blindless_AIFFELTON/Ablation Study/")

# import customized modules
import preprocess, data_generator, check_result, metrics, loss, train

sys.path.append("/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid")
import pyramid

# Insert Path
# {model_name}_{epoch}ep_{model_inpusize}_{Generator | aG/fG/sG}
FILE_NAME = 'pyramid_30ep_1024_fG_wo_decay'
INPUT_SHAPE = (1024, 1024, 1)
MODEL_PATH = "/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/" + FILE_NAME + ".tf"
HISTORY_PATH = "/visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/history/" + FILE_NAME + '_history' ".json"

TRAIN_SOURCE = '/visuworks/Dataset/Selected Dataset 3/Train'
TEST_SOURCE = '/visuworks/Dataset/Selected Dataset 3/Test'

2024-01-31 07:44:29.264009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 07:44:29.264037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 07:44:29.265133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 07:44:29.270461: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-31 07:44:29.843832: W tensorflow/compiler/tf2

In [2]:
# Define custom objects for loading the model
custom_objects = {'DiceLoss': loss.DiceLoss(), 
                  'sensitivity': metrics.sensitivity,
                  'specificity': metrics.specificity,
                  'accuracy' : metrics.accuracy,}

# Create augmentation
train_preproc = preprocess.build_augmentation_for_general(INPUT_SHAPE[0], INPUT_SHAPE[0])
test_preproc = preprocess.build_augmentation_for_general(INPUT_SHAPE[0], INPUT_SHAPE[0],is_train=False)

# Create train, test generator
train_generator = data_generator.FullSizedDataGenerator(
    TRAIN_SOURCE, 
    batch_size=1, 
    img_size=INPUT_SHAPE,
    output_size=INPUT_SHAPE,
    is_train=True,
    augmentation= train_preproc
)

test_generator = data_generator.FullSizedDataGenerator(
    TRAIN_SOURCE, 
    batch_size=1, 
    img_size=INPUT_SHAPE,
    output_size=INPUT_SHAPE,
    is_train=False,
    augmentation= test_preproc
)

# Create model
model = pyramid.build_model(INPUT_SHAPE)
model.summary()

2024-01-31 07:44:31.006826: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 07:44:31.042202: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 07:44:31.042427: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1024, 1024, 1)]      0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 1024, 1024, 16)       32        ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 1024, 1024, 8)        16        ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 1024, 1024, 16)       160       ['input_1[0][0]']             
                                                                                              

In [3]:
train.model_train(model,
                  epoch = 30,
                  train_generator = train_generator,
                  test_generator = test_generator,
                  model_path = MODEL_PATH,
                  history_path = HISTORY_PATH,
                  save_every_epoch=True,
                  apply_weight_decay=False)

Epoch 1/30


2024-01-31 07:44:36.041551: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-31 07:44:39.153066: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fb84fa500a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-31 07:44:39.153089: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2024-01-31 07:44:39.156472: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706687079.229101 1082436 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


879/880 [============================>.] - ETA: 0s - loss: 0.4689 - sensitivity: 0.4545 - specificity: 0.9680 - accuracy: 0.9331INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/1--0.4687-0.4548-0.9680-0.5994-0.9727.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/1--0.4687-0.4548-0.9680-0.5994-0.9727.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/1--0.4687-0.4548-0.9680-0.5994-0.9727.pb
880/880 [==============================] - 142s 141ms/step - loss: 0.4687 - sensitivity: 0.4548 - specificity: 0.9680 - accuracy: 0.9331 - val_loss: 0.3459 - val_sensitivity: 0.5994 - val_specificity: 0.9727 - val_accuracy: 0.9478
Epoch 2/30
880/880 [==============================] - ETA: 0s - loss: 0.3276 - sensitivity: 0.6487 - specificity: 0.9695 - accuracy: 0.9484INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/2--0.3276-0.6487-0.9695-0.6806-0.9695.pb/assets


...

Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/28--0.2170-0.7626-0.9809-0.7113-0.9847.pb
880/880 [==============================] - 117s 133ms/step - loss: 0.2170 - sensitivity: 0.7626 - specificity: 0.9809 - accuracy: 0.9668 - val_loss: 0.2266 - val_sensitivity: 0.7113 - val_specificity: 0.9847 - val_accuracy: 0.9666
Epoch 29/30
880/880 [==============================] - ETA: 0s - loss: 0.2160 - sensitivity: 0.7629 - specificity: 0.9811 - accuracy: 0.9670INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/29--0.2160-0.7629-0.9811-0.7637-0.9796.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/29--0.2160-0.7629-0.9811-0.7637-0.9796.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/29--0.2160-0.7629-0.9811-0.7637-0.9796.pb
880/880 [==============================] - 115s 131ms/step - loss: 0.2160 - sensitivity: 0.7629 - specificity: 0.9811 - accuracy: 0.9670 - val_loss: 0.2215 - val_sensitivity: 0.7637 - val_specificity: 0.9796 - val_accuracy: 0.9657
Epoch 30/30
879/880 [============================>.] - ETA: 0s - loss: 0.2144 - sensitivity: 0.7646 - specificity: 0.9812 - accuracy: 0.9672INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/30--0.2146-0.7646-0.9812-0.7205-0.9843.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/30--0.2146-0.7646-0.9812-0.7205-0.9843.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Pyramid/model_parameters/pyramid_30ep_1024_fG_wo_decay.tf/30--0.2146-0.7646-0.9812-0.7205-0.9843.pb
880/880 [==============================] - 116s 131ms/step - loss: 0.2146 - sensitivity: 0.7646 - specificity: 0.9812 - accuracy: 0.9671 - val_loss: 0.2223 - val_sensitivity: 0.7205 - val_specificity: 0.9843 - val_accuracy: 0.9669
